In [5]:
import numpy as np
import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Layer
from keras.optimizers import RMSprop
from keras import backend as K
from keras.utils.vis_utils import plot_model, model_to_dot
from IPython.display import SVG
from matplotlib import pyplot as plt

In [8]:
def euclidean_distance(vects):
    x,y = vects
    sum_square = K.sum(K.square(x-y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [ ]:
from nltk.tokenize import word_tokenize
import numpy as np
import pickle as pkl

with open('./data/dwords.p', 'rb') as f:
    dtr = pkl.load(f)


def load_set(embed, datapath, embed_dim):
    with open(datapath, 'rb') as f:
        mylist = pkl.load(f)
    s0 = []
    s1 = []
    labels = []
    for each in mylist:
        s0x = each[0]
        s1x = each[1]
        label = each[2]
        score = float(label)
        labels.append(score)
        for i, ss in enumerate([s0x, s1x]):
            words = word_tokenize(ss)
            index = []
            for word in words:
                if word in dtr:
                    index.append(embed[dtr[word]])
                elif word in embed.vocab:
                    index.append(embed[word])
                else:
                    index.append(np.zeros(embed_dim, dtype=float))

            if i == 0:
                s0.append(np.array(index, dtype=float))
            else:
                s1.append(np.array(index, dtype=float))

    return [s0, s1, labels]



def load_data(max_len, embed, datapath, embed_dim):
    train_set = load_set(embed, datapath, embed_dim)
    train_set_x1, train_set_x2, train_set_y = train_set
    # train_set length
    n_samples = len(train_set_x1)

    
    sidx = np.random.permutation(n_samples)

    train_set_x1 = [train_set_x1[s] for s in sidx]
    train_set_x2 = [train_set_x2[s] for s in sidx]
    train_set_y = [train_set_y[s] for s in sidx]

    
    train_set = [train_set_x1, train_set_x2, train_set_y]

    
    new_train_set_x1 = np.zeros([len(train_set[0]), max_len, embed_dim], dtype=float)
    new_train_set_x2 = np.zeros([len(train_set[0]), max_len, embed_dim], dtype=float)
    new_train_set_y = np.zeros(len(train_set[0]), dtype=float)
    mask_train_x1 = np.zeros([len(train_set[0]), max_len])
    mask_train_x2 = np.zeros([len(train_set[0]), max_len])

    def padding_and_generate_mask(x1, x2, y, new_x1, new_x2, new_y, new_mask_x1, new_mask_x2):

        for i, (x1, x2, y) in enumerate(zip(x1, x2, y)):
            # whether to remove sentences with length larger than maxlen
            if len(x1) <= max_len:
                new_x1[i, 0:len(x1)] = x1
                new_mask_x1[i, len(x1) - 1] = 1
                new_y[i] = y
            else:
                new_x1[i, :, :] = (x1[0:max_len:embed_dim])
                new_mask_x1[i, max_len - 1] = 1
                new_y[i] = y
            if len(x2) <= max_len:
                new_x2[i, 0:len(x2)] = x2
                new_mask_x2[i, len(x2) - 1] = 1  
                new_y[i] = y
            else:
                new_x2[i, :, :] = (x2[0:max_len:embed_dim])
                new_mask_x2[i, max_len - 1] = 1
                new_y[i] = y
        new_set = [new_x1, new_x2, new_y, new_mask_x1, new_mask_x2]
        del new_x1, new_x2, new_y
        return new_set

    train_set = padding_and_generate_mask(train_set[0], train_set[1], train_set[2], new_train_set_x1, new_train_set_x2,
                                          new_train_set_y, mask_train_x1, mask_train_x2)

    return train_set


# return batch dataset
def batch_iter(data, batch_size):
    # get dataset and label
    x1, x2, y, mask_x1, mask_x2 = data
    x1 = np.array(x1)
    x2 = np.array(x2)
    y = np.array(y)
    data_size = len(x1)
    num_batches_per_epoch = int((data_size - 1) / batch_size) + 1
    for batch_index in range(num_batches_per_epoch):
        start_index = batch_index * batch_size
        end_index = min((batch_index + 1) * batch_size, data_size)
        return_x1 = x1[start_index:end_index]
        return_x2 = x2[start_index:end_index]
        return_y = y[start_index:end_index]
        return_mask_x1 = mask_x1[start_index:end_index]
        return_mask_x2 = mask_x2[start_index:end_index]

        yield [return_x1, return_x2, return_y, return_mask_x1, return_mask_x2]

In [ ]:
class LinearSeqAttn(nn.Module):
    """Self attention over a sequence:
    * o_i = softmax(Wx_i) for x_i in X.
    """
    def __init__(self, input_size):
        super(LinearSeqAttn, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x, x_mask):
        """
        x = batch * len * hdim
        x_mask = batch * len
        """
        x_flat = x.contiguous().view(-1, x.size(-1))
        scores = self.linear(x_flat).view(x.size(0), x.size(1))
        scores.data.masked_fill_(x_mask.data, -float('inf'))
        alpha = F.softmax(scores, dim=1)
        return alpha


In [2]:
class BilinearSeqAttn(nn.Module):
    """A bilinear attention layer over a sequence X w.r.t y:
    * o_i = softmax(x_i'Wy) for x_i in X.

    Optionally don't normalize output weights.
    """
    def __init__(self, x_size, y_size, identity=False):
        super(BilinearSeqAttn, self).__init__()
        if not identity:
            self.linear = nn.Linear(y_size, x_size)
        else:
            self.linear = None

    def forward(self, x, y, x_mask):
        """
        x = batch * len * h1
        y = batch * h2
        x_mask = batch * len
        """
        Wy = self.linear(y) if self.linear is not None else y
        xWy = x.bmm(Wy.unsqueeze(2)).squeeze(2)
        xWy.data.masked_fill_(x_mask.data, -float('inf'))
        if self.training:
            # In training we output log-softmax for NLL
            alpha = F.log_softmax(xWy, dim=1)
        else:
            # ...Otherwise 0-1 probabilities
            alpha = F.softmax(xWy, dim=1)
        return alpha

NameError: name 'nn' is not defined

In [3]:
abc = load_set

NameError: name 'load_set' is not defined